In [2]:
%pip install -q --upgrade keras keras-cv keras_tuner tensorflow setuptools scikit-learn segmentation-models matplotlib opencv-python seaborn pandas
print("Successfully installed required packages.")


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [16]:
%pip install kaggle
!kaggle datasets download -d tawsifurrahman/tuberculosis-tb-chest-xray-dataset
!kaggle datasets download -d nikhilpandey360/chest-xray-masks-and-labels
!unzip chest-xray-masks-and-labels.zip
!unzip tuberculosis-tb-chest-xray-dataset.zip
!rm -rf data
!mv Lung\ Segmentation segmentation_data
!rm -rf ./segmentation_data/ClinicalReadings
!rm ./segmentation_data/NLM-ChinaCXRSet-ReadMe.docx
!rm ./segmentation_data/NLM-MontgomeryCXRSet-ReadMe.pdf
!mv ./segmentation_data/CXR_png ./segmentation_data/images
!mv TB_Chest_Radiography_Database data

Dataset URL: https://www.kaggle.com/datasets/tawsifurrahman/tuberculosis-tb-chest-xray-dataset
License(s): copyright-authors
100%|███████████████████████████████████████▉| 663M/663M [00:23<00:00, 33.7MB/s]
100%|████████████████████████████████████████| 663M/663M [00:23<00:00, 29.2MB/s]
Archive:  tuberculosis-tb-chest-xray-dataset.zip
  inflating: TB_Chest_Radiography_Database/Normal.metadata.xlsx  
  inflating: TB_Chest_Radiography_Database/Normal/Normal-1.png  
  inflating: TB_Chest_Radiography_Database/Normal/Normal-10.png  
  inflating: TB_Chest_Radiography_Database/Normal/Normal-100.png  
  inflating: TB_Chest_Radiography_Database/Normal/Normal-1000.png  
  inflating: TB_Chest_Radiography_Database/Normal/Normal-1001.png  
  inflating: TB_Chest_Radiography_Database/Normal/Normal-1002.png  
  inflating: TB_Chest_Radiography_Database/Normal/Normal-1003.png  
  inflating: TB_Chest_Radiography_Database/Normal/Normal-1004.png  
  inflating: TB_Chest_Radiography_Database/Normal/Normal-100

In [4]:
!apt-get update && apt-get install ffmpeg libsm6 libxext6  -y

0% [Working]

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease                         
Hit:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease              
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  alsa-topology-conf alsa-ucm-conf fontconfig fontconfig-config
  fonts-dejavu-core i965-va-driver intel-media-va-driver libaacs0 libaom3
  libasound2 libasound2-data libass9 libasyncns0 libavc1394-0 libavcodec58
  libavdevice58 libavfilter7 libavformat58 libavutil56 libbdplus0 libblas3
  libbluray2 libbs2b0 libcaca0 libcairo-gobject2 libcairo2 libcdio-cdda2
  libcdio-paranoia2 libcdio19 libchromaprint1 libcodec2-1.0 libdatrie1
  libdav1

In [1]:
import os
os.environ["SM_FRAMEWORK"] = "tf.keras"

import segmentation_models as sm
import keras
import keras_tuner
import keras_cv
from sklearn.model_selection import train_test_split
import shutil
import matplotlib.pyplot as plt
import os
import numpy as np
import tensorflow as tf
import cv2
# import kfold

2024-08-29 12:55:18.463757: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-29 12:55:18.476043: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-29 12:55:18.490111: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-29 12:55:18.494158: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-29 12:55:18.504782: I tensorflow/core/platform/cpu_feature_guar

Segmentation Models: using `tf.keras` framework.


/usr/local/lib/python3.11/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
tf.config.set_visible_devices([], 'GPU')

In [7]:
DATA_DIR = "./segmentation_data"
image_dir = os.path.join(DATA_DIR, "images")
mask_dir = os.path.join(DATA_DIR, "masks")

masks = [f for f in os.listdir(mask_dir) if os.path.isfile(os.path.join(mask_dir, f))]
for mask in masks:
    if mask.endswith("_mask.png"):
        os.rename(os.path.join(mask_dir, mask), os.path.join(mask_dir, mask.replace("_mask", "")))

images = [f for f in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, f))]
for img in images:
    if os.path.isfile(os.path.join(mask_dir, img)):
        continue
    os.remove(os.path.join(image_dir, img))

# Split the dataset into training and validation sets
images = [f for f in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, f))]
masks = [f for f in os.listdir(mask_dir) if os.path.isfile(os.path.join(mask_dir, f))]

images = [f for f in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, f))]
masks = [f for f in os.listdir(mask_dir) if os.path.isfile(os.path.join(mask_dir, f))]

train_images, val_images, train_masks, val_masks = train_test_split(images, masks, test_size=0.1)

if os.path.exists(os.path.join(DATA_DIR, "train")):
    print("Removing existing train dataset...")
    shutil.rmtree(os.path.join(DATA_DIR, "train"))

if os.path.exists(os.path.join(DATA_DIR, "val")):
    print("Removing existing validation dataset...")
    shutil.rmtree(os.path.join(DATA_DIR, "val"))

train_image_dir = os.path.join(DATA_DIR, "train", "images")
val_image_dir = os.path.join(DATA_DIR, "val", "images")
train_mask_dir = os.path.join(DATA_DIR, "train", "masks")
val_mask_dir = os.path.join(DATA_DIR, "val", "masks")

os.makedirs(train_image_dir, exist_ok=True)
os.makedirs(val_image_dir, exist_ok=True)
os.makedirs(train_mask_dir, exist_ok=True)
os.makedirs(val_mask_dir, exist_ok=True)

def copy_files(file_list, src_dir, dst_dir):
    for file in file_list:
        shutil.copy(os.path.join(src_dir, file), os.path.join(dst_dir, file))

print("Copying train images...")
copy_files(train_images, image_dir, train_image_dir)
print("Copying validation images...")
copy_files(val_images, image_dir, val_image_dir)
print("Copying train masks...")
copy_files(train_masks, mask_dir, train_mask_dir)
print("Copying validation masks...")
copy_files(val_masks, mask_dir, val_mask_dir)

Copying train images...
Copying validation images...
Copying train masks...
Copying validation masks...


In [2]:
# https://www.tensorflow.org/tutorials/images/segmentation
class Augment(tf.keras.layers.Layer):
    def __init__(self):
        super().__init__()

        self.augment_inputs = keras.Sequential([
            keras_cv.layers.RandomColorJitter(
                value_range=(0, 255),
                brightness_factor=0.2,
                contrast_factor=0.2,
                saturation_factor=0.2,
                hue_factor=0.2,
            )
        ])

    def call(self, inputs, labels):
        inputs = self.augment_inputs(inputs)

        return inputs, labels / 255

def augment_validation(inputs, labels):
    return inputs, labels / 255

def get_data(batch_size, base_img_dir='segmentation_data/train', augmentation=True, image_size=(256, 256), masks_size=(256, 256)):
    train_images, validation_images = keras.utils.image_dataset_from_directory(
        base_img_dir + '/images',
        labels=None,
        batch_size=batch_size,
        image_size=image_size,
        shuffle=False,
        validation_split=0.1,
        subset='both'
    )

    train_masks, validation_masks = keras.utils.image_dataset_from_directory(
        base_img_dir + '/masks',
        labels=None,
        batch_size=batch_size,
        image_size=masks_size,
        color_mode='grayscale',
        shuffle=False,
        validation_split=0.1,
        subset='both'
    )

    training_dataset = tf.data.Dataset.zip(train_images, train_masks)
    validation_dataset = tf.data.Dataset.zip(validation_images, validation_masks)

    if augmentation == True:
        training_dataset = training_dataset.map(Augment()).prefetch(buffer_size=tf.data.AUTOTUNE)
    else:
        training_dataset = training_dataset.map(augment_validation).prefetch(buffer_size=tf.data.AUTOTUNE)
    validation_dataset = validation_dataset.map(augment_validation).prefetch(buffer_size=tf.data.AUTOTUNE)

    return training_dataset, validation_dataset

## Resnet34

In [8]:
BACKBONE = 'resnet34'
def build_resnet_model(hp):
    loss_idx = hp.Choice('loss_idx', values=[0, 1])
    losses = [keras.losses.binary_crossentropy, sm.losses.dice_loss]
    loss = losses[loss_idx]

    optimizer_idx = hp.Choice('optimizer_idx', values=[0, 1])
    optimizers = [keras.optimizers.Adam(), keras.optimizers.SGD()]
    optimizer = optimizers[optimizer_idx]

    model = sm.Unet(BACKBONE, encoder_weights='imagenet', encoder_freeze=True)

    model.compile(
        optimizer=optimizer,
        loss=loss,
        metrics=['accuracy', sm.metrics.IOUScore(), sm.metrics.FScore()]
    )

    return model

In [9]:
preprocess_input = sm.get_preprocessing(BACKBONE)

def process_dataset(images, labels):
    return preprocess_input(images), labels

for batch_size in [4, 8, 16]:
    project_name= 'resnet34-%d' % batch_size

    training_dataset, validation_dataset = get_data(batch_size)
    training_dataset = training_dataset.map(process_dataset).prefetch(buffer_size=tf.data.AUTOTUNE)
    validation_dataset = validation_dataset.map(process_dataset).prefetch(buffer_size=tf.data.AUTOTUNE)

    tuner = keras_tuner.GridSearch(
        objective='val_loss',
        hypermodel=build_resnet_model,
        project_name='models/' + project_name
    )

    tensorboard_cb = keras.callbacks.TensorBoard(log_dir='tensorboard/%s' % (project_name))
    early_stopping_cb = keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True, start_from_epoch=10)
    reduce_lr_cb = keras.callbacks.ReduceLROnPlateau(patience=5, cooldown=5)

    tuner.search(
        training_dataset,
        batch_size=batch_size,
        epochs=50,
        validation_data=validation_dataset,
        callbacks=[tensorboard_cb, early_stopping_cb, reduce_lr_cb]
    )

Found 633 files.
Using 570 files for training.
Using 63 files for validation.
Found 633 files.
Using 570 files for training.
Using 63 files for validation.
Reloading Tuner from ./models/resnet34-4/tuner0.json
Found 633 files.
Using 570 files for training.
Using 63 files for validation.
Found 633 files.
Using 570 files for training.
Using 63 files for validation.
Reloading Tuner from ./models/resnet34-8/tuner0.json
Found 633 files.
Using 570 files for training.
Using 63 files for validation.
Found 633 files.
Using 570 files for training.
Using 63 files for validation.
Reloading Tuner from ./models/resnet34-16/tuner0.json


## Inceptionv3

In [10]:
BACKBONE = 'inceptionv3'
def build_inceptionv3_model(hp):
    loss_idx = hp.Choice('loss_idx', values=[0, 1])
    losses = [keras.losses.binary_crossentropy, sm.losses.dice_loss]
    loss = losses[loss_idx]

    optimizer_idx = hp.Choice('optimizer_idx', values=[0, 1])
    optimizers = [keras.optimizers.Adam(), keras.optimizers.SGD()]
    optimizer = optimizers[optimizer_idx]

    model = sm.Unet(BACKBONE, encoder_weights='imagenet', encoder_freeze=True)

    model.compile(
        optimizer=optimizer,
        loss=loss,
        metrics=['accuracy', sm.metrics.IOUScore(), sm.metrics.FScore()]
    )

    return model

In [11]:
preprocess_input = sm.get_preprocessing(BACKBONE)

def process_dataset(images, labels):
    return preprocess_input(images), labels

for batch_size in [4, 8, 16]:
    project_name = 'inceptionv3-%d' % batch_size

    training_dataset, validation_dataset = get_data(batch_size)
    training_dataset = training_dataset.map(process_dataset).prefetch(buffer_size=tf.data.AUTOTUNE)
    validation_dataset = validation_dataset.map(process_dataset).prefetch(buffer_size=tf.data.AUTOTUNE)

    tuner = keras_tuner.GridSearch(
        objective='val_loss',
        hypermodel=build_inceptionv3_model,
        project_name='models/' + project_name
    )

    tensorboard_cb = keras.callbacks.TensorBoard(log_dir='tensorboard/%s' % (project_name))
    early_stopping_cb = keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True, start_from_epoch=10)
    reduce_lr_cb = keras.callbacks.ReduceLROnPlateau(patience=5, cooldown=5)

    tuner.search(
        training_dataset,
        batch_size=batch_size,
        epochs=50,
        validation_data=validation_dataset,
        callbacks=[tensorboard_cb, early_stopping_cb, reduce_lr_cb]
    )

Found 633 files.
Using 570 files for training.
Using 63 files for validation.
Found 633 files.
Using 570 files for training.
Using 63 files for validation.
Reloading Tuner from ./models/inceptionv3-4/tuner0.json
Found 633 files.
Using 570 files for training.
Using 63 files for validation.
Found 633 files.
Using 570 files for training.
Using 63 files for validation.
Reloading Tuner from ./models/inceptionv3-8/tuner0.json
Found 633 files.
Using 570 files for training.
Using 63 files for validation.
Found 633 files.
Using 570 files for training.
Using 63 files for validation.
Reloading Tuner from ./models/inceptionv3-16/tuner0.json


## Evaluating InceptionV3

In [4]:
BACKBONE = 'inceptionv3'
evaluations = []
histories = []

preprocess_input = sm.get_preprocessing(BACKBONE)

def process_dataset(images, labels):
    return preprocess_input(images), labels

split, image_names, mask_names = kfold.fold_segmentation_dataset()
for i, (train_idx, test_idx) in enumerate(split):
    batch_size = 4
    project_name = 'inception-evaluation/fold-%d' % (i + 1)
    train_images = image_names[train_idx]
    train_masks = mask_names[train_idx]
    test_images = image_names[test_idx]
    test_masks = mask_names[test_idx]

    print('Splitting for fold %d' % i)
    kfold.copy_segmentation_dataset_to_folds({'images': train_images, 'masks': train_masks}, {'images': test_images, 'masks': test_masks})

    training_dataset, validation_dataset = get_data(batch_size, base_img_dir='segmentation_data/folds/train')
    training_dataset = training_dataset.map(process_dataset).prefetch(buffer_size=tf.data.AUTOTUNE)
    validation_dataset = validation_dataset.map(process_dataset).prefetch(buffer_size=tf.data.AUTOTUNE)

    model = sm.Unet(BACKBONE, encoder_weights='imagenet', encoder_freeze=True)
    model.compile(
        optimizer=keras.optimizers.Adam(),
        loss=sm.losses.dice_loss,
        metrics=['accuracy', sm.metrics.IOUScore(), sm.metrics.FScore()]
    )

    # train the decoder
    model.fit(
        training_dataset,
        validation_data=validation_dataset,
        epochs=3
    )

    # Allow the encoder weights to be updated, at a smaller learning rate
    for layer in model.layers:
        layer.trainable = True

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.0001),
        loss=sm.losses.dice_loss,
        metrics=['accuracy', sm.metrics.IOUScore(), sm.metrics.FScore()]
    )

    tensorboard_cb = keras.callbacks.TensorBoard(log_dir='tensorboard/%s' % (project_name))
    early_stopping_cb = keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
    reduce_lr_cb = keras.callbacks.ReduceLROnPlateau(patience=5, cooldown=5)

    history = model.fit(
        training_dataset,
        validation_data=validation_dataset,
        epochs=50,
        callbacks=[tensorboard_cb, early_stopping_cb, reduce_lr_cb]
    )

    test_images = keras.utils.image_dataset_from_directory(
        'segmentation_data/folds/test' + '/images',
        labels=None,
        batch_size=batch_size,
        image_size=(256, 256),
        shuffle=False
    )

    test_masks = keras.utils.image_dataset_from_directory(
        'segmentation_data/folds/test' + '/masks',
        labels=None,
        batch_size=batch_size,
        image_size=(256, 256),
        color_mode='grayscale',
        shuffle=False
    )

    test_dataset = tf.data.Dataset.zip(test_images, test_masks)
    test_dataset = test_dataset.map(augment_validation).prefetch(buffer_size=tf.data.AUTOTUNE)
    test_dataset = test_dataset.map(process_dataset).prefetch(buffer_size=tf.data.AUTOTUNE)

    histories.append(history)
    evaluations.append(model.evaluate(test_dataset))


Splitting for fold 0
Creating fold dirs...
Copying the training data...
Copying the test data...
Found 563 files.
Using 507 files for training.
Using 56 files for validation.
Found 563 files.
Using 507 files for training.
Using 56 files for validation.
Epoch 1/3
127/127 ━━━━━━━━━━━━━━━━━━━━ 134s 882ms/step - accuracy: 0.9102 - f1-score: 0.7571 - iou_score: 0.6374 - loss: 0.2429 - val_accuracy: 0.9550 - val_f1-score: 0.9039 - val_iou_score: 0.8248 - val_loss: 0.0961
Epoch 2/3
127/127 ━━━━━━━━━━━━━━━━━━━━ 114s 893ms/step - accuracy: 0.9604 - f1-score: 0.9315 - iou_score: 0.8724 - loss: 0.0685 - val_accuracy: 0.9751 - val_f1-score: 0.9514 - val_iou_score: 0.9075 - val_loss: 0.0486
Epoch 3/3
127/127 ━━━━━━━━━━━━━━━━━━━━ 115s 897ms/step - accuracy: 0.9626 - f1-score: 0.9400 - iou_score: 0.8874 - loss: 0.0600 - val_accuracy: 0.9785 - val_f1-score: 0.9595 - val_iou_score: 0.9223 - val_loss: 0.0405
Epoch 1/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 161s 972ms/step - accuracy: 0.9651 - f1-score: 0.9461 - 

In [6]:
np_evals = np.array(evaluations)
evaluation_means = np.mean(np_evals, axis=0)

for i, evaluation in enumerate(np_evals):
    print('Fold %d - Loss: %f; Accuracy: %f; IoU: %f; Dice: %f' % (i + 1, evaluation[0], evaluation[1], evaluation[2], evaluation[3]))

print('--- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- ---')
print('Aggregated Results - Inceptionv3 Style K-Fold Evaluation:')
for (mean_score, metric_name) in zip(evaluation_means, ['Loss', 'Accuracy', 'IoU', 'Dice']):
    print('%s: %f' % (metric_name, mean_score))
print('Average Number of Epochs: %f' % 11.4)
print('Average Training Time: %f minutes' % 26.4)

Fold 1 - Loss: 0.048405; Accuracy: 0.975351; IoU: 0.909015; Dice: 0.952065
Fold 2 - Loss: 0.045762; Accuracy: 0.976850; IoU: 0.913833; Dice: 0.954674
Fold 3 - Loss: 0.039111; Accuracy: 0.979012; IoU: 0.925799; Dice: 0.961234
Fold 4 - Loss: 0.037808; Accuracy: 0.980125; IoU: 0.928108; Dice: 0.962573
Fold 5 - Loss: 0.041074; Accuracy: 0.978712; IoU: 0.921605; Dice: 0.958926
--- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- ---
Aggregated Results - Inceptionv3 Style K-Fold Evaluation:
Loss: 0.042432
Accuracy: 0.978010
IoU: 0.919672
Dice: 0.957895
Average Number of Epochs: 11.400000
Average Training Time: 26.400000 minutes


![Tensorboard Inceptionv3 Loss Evaluation](./assets/inceptionv3-eval/epoch-loss-time.png)
![Tensorboard Inceptionv3 Loss Curve](./assets/inceptionv3-eval/epoch-loss-curve.png)

## XCeption Style

In [12]:
# TODO:: CITE
def build_xception(img_size, dropout_rate=0.0):
    inputs = keras.Input(shape=img_size + (3,))

    ### [First half of the network: downsampling inputs] ###

    # Entry block
    x = keras.layers.Conv2D(32, 3, strides=2, padding="same")(inputs)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    # Blocks 1, 2, 3 are identical apart from the feature depth.
    for filters in [64, 128, 256]:
        x = keras.layers.Activation("relu")(x)
        x = keras.layers.SeparableConv2D(filters, 3, padding="same")(x)
        x = keras.layers.BatchNormalization()(x)

        x = keras.layers.Activation("relu")(x)
        x = keras.layers.SeparableConv2D(filters, 3, padding="same")(x)
        x = keras.layers.BatchNormalization()(x)

        x = keras.layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = keras.layers.Conv2D(filters, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = keras.layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    ### [Second half of the network: upsampling inputs] ###

    for filters in [256, 128, 64, 32]:
        x = keras.layers.Activation("relu")(x)
        x = keras.layers.Conv2DTranspose(filters, 3, padding="same")(x)
        x = keras.layers.BatchNormalization()(x)

        x = keras.layers.Activation("relu")(x)
        x = keras.layers.Conv2DTranspose(filters, 3, padding="same")(x)
        x = keras.layers.BatchNormalization()(x)

        x = keras.layers.UpSampling2D(2)(x)

        # Project residual
        residual = keras.layers.UpSampling2D(2)(previous_block_activation)
        residual = keras.layers.Conv2D(filters, 1, padding="same")(residual)
        x = keras.layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    dropout = keras.layers.Dropout(dropout_rate)(x)
    # Add a per-pixel classification layer
    outputs = keras.layers.Conv2D(1, 3, activation="sigmoid", padding="same")(dropout)

    # Define the model
    model = keras.Model(inputs, outputs)
    return model


def build_xception_model(hp):
    loss_idx = hp.Choice('loss_idx', values=[0, 1])
    losses = [keras.losses.binary_crossentropy, sm.losses.dice_loss]
    loss = losses[loss_idx]

    optimizer_idx = hp.Choice('optimizer_idx', values=[0, 1])
    optimizers = [keras.optimizers.Adam(), keras.optimizers.SGD()]
    optimizer = optimizers[optimizer_idx]

    model = build_xception((256, 256))

    model.compile(
        optimizer=optimizer,
        loss=loss,
        metrics=['accuracy', sm.metrics.IOUScore(), sm.metrics.FScore()]
    )

    return model

def preprocess_input(images):
    return images / 255

def process_dataset(images, labels):
    return preprocess_input(images), labels

In [13]:
def preprocess_input(images):
    return images / 255

def process_dataset(images, labels):
    return preprocess_input(images), labels

for batch_size in [4, 8, 16]:
    project_name = 'xception-%d' % batch_size

    training_dataset, validation_dataset = get_data(batch_size)
    training_dataset = training_dataset.map(process_dataset).prefetch(buffer_size=tf.data.AUTOTUNE)
    validation_dataset = validation_dataset.map(process_dataset).prefetch(buffer_size=tf.data.AUTOTUNE)

    tuner = keras_tuner.GridSearch(
        objective='val_loss',
        hypermodel=build_xception_model,
        project_name='models/' + project_name
    )

    tensorboard_cb = keras.callbacks.TensorBoard(log_dir='tensorboard/%s' % (project_name))
    early_stopping_cb = keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
    reduce_lr_cb = keras.callbacks.ReduceLROnPlateau(patience=5, cooldown=5)

    tuner.search(
        training_dataset,
        batch_size=batch_size,
        epochs=50,
        validation_data=validation_dataset,
        callbacks=[tensorboard_cb, early_stopping_cb, reduce_lr_cb]
    )

Found 633 files.
Using 570 files for training.
Using 63 files for validation.
Found 633 files.
Using 570 files for training.
Using 63 files for validation.
Reloading Tuner from ./models/xception-4/tuner0.json
Found 633 files.
Using 570 files for training.
Using 63 files for validation.
Found 633 files.
Using 570 files for training.
Using 63 files for validation.
Reloading Tuner from ./models/xception-8/tuner0.json
Found 633 files.
Using 570 files for training.
Using 63 files for validation.
Found 633 files.
Using 570 files for training.
Using 63 files for validation.
Reloading Tuner from ./models/xception-16/tuner0.json


In [21]:
evaluations = []
histories = []

def preprocess_input(images):
    return images / 255

def process_dataset(images, labels):
    return preprocess_input(images), labels

split, image_names, mask_names = kfold.fold_segmentation_dataset()
for i, (train_idx, test_idx) in enumerate(split):
    batch_size = 4
    project_name = 'xception-evaluation/fold-%d' % (i + 1)
    train_images = image_names[train_idx]
    train_masks = mask_names[train_idx]
    test_images = image_names[test_idx]
    test_masks = mask_names[test_idx]

    print('Splitting for fold %d' % i)
    kfold.copy_segmentation_dataset_to_folds({'images': train_images, 'masks': train_masks}, {'images': test_images, 'masks': test_masks})

    training_dataset, validation_dataset = get_data(batch_size, base_img_dir='segmentation_data/folds/train')
    training_dataset = training_dataset.map(process_dataset).prefetch(buffer_size=tf.data.AUTOTUNE)
    validation_dataset = validation_dataset.map(process_dataset).prefetch(buffer_size=tf.data.AUTOTUNE)

    model = build_xception((256, 256))
    model.compile(
        optimizer=keras.optimizers.Adam(),
        loss=sm.losses.dice_loss,
        metrics=['accuracy', sm.metrics.IOUScore(), sm.metrics.FScore()]
    )

    tensorboard_cb = keras.callbacks.TensorBoard(log_dir='tensorboard/%s' % (project_name))
    early_stopping_cb = keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
    reduce_lr_cb = keras.callbacks.ReduceLROnPlateau(patience=5, cooldown=5)

    history = model.fit(
        training_dataset,
        validation_data=validation_dataset,
        epochs=50,
        callbacks=[tensorboard_cb, early_stopping_cb, reduce_lr_cb]
    )

    test_images = keras.utils.image_dataset_from_directory(
        'segmentation_data/folds/test' + '/images',
        labels=None,
        batch_size=batch_size,
        image_size=(256, 256),
        shuffle=False
    )

    test_masks = keras.utils.image_dataset_from_directory(
        'segmentation_data/folds/test' + '/masks',
        labels=None,
        batch_size=batch_size,
        image_size=(256, 256),
        color_mode='grayscale',
        shuffle=False
    )

    test_dataset = tf.data.Dataset.zip(test_images, test_masks)
    test_dataset = test_dataset.map(augment_validation).prefetch(buffer_size=tf.data.AUTOTUNE)
    test_dataset = test_dataset.map(process_dataset).prefetch(buffer_size=tf.data.AUTOTUNE)

    histories.append(history)
    evaluations.append(model.evaluate(test_dataset))


Splitting for fold 0
Creating fold dirs...
Copying the training data...
Copying the test data...
Found 563 files.
Using 507 files for training.
Using 56 files for validation.
Found 563 files.
Using 507 files for training.
Using 56 files for validation.
Epoch 1/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 65s 397ms/step - accuracy: 0.8123 - f1-score: 0.7257 - iou_score: 0.5916 - loss: 0.2743 - val_accuracy: 0.8591 - val_f1-score: 0.5171 - val_iou_score: 0.3491 - val_loss: 0.4829 - learning_rate: 0.0010
Epoch 2/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 54s 403ms/step - accuracy: 0.9429 - f1-score: 0.8988 - iou_score: 0.8170 - loss: 0.1012 - val_accuracy: 0.7314 - val_f1-score: 1.5441e-10 - val_iou_score: 1.4921e-10 - val_loss: 1.0000 - learning_rate: 0.0010
Epoch 3/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 55s 424ms/step - accuracy: 0.9478 - f1-score: 0.9095 - iou_score: 0.8345 - loss: 0.0905 - val_accuracy: 0.7314 - val_f1-score: 8.1568e-10 - val_iou_score: 4.7984e-10 - val_loss: 1.0000 - learning_rate: 0.0010
Epoch 4/

In [47]:
np_evals = np.array(evaluations)
evaluation_means = np.mean(np_evals, axis=0)

for i, evaluation in enumerate(np_evals):
    print('Fold %d - Loss: %f; Accuracy: %f; IoU: %f; Dice: %f' % (i + 1, evaluation[0], evaluation[1], evaluation[2], evaluation[3]))

print('--- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- ---')
print('Aggregated Results - Xception Style K-Fold Evaluation:')
for (mean_score, metric_name) in zip(evaluation_means, ['Loss', 'Accuracy', 'IoU', 'Dice']):
    print('%s: %f' % (metric_name, mean_score))
print('Average Number of Epochs: %f' % 18.4)
print('Average Training Time: %f minutes' % 17.17)

Fold 1 - Loss: 0.069888; Accuracy: 0.964547; IoU: 0.870696; Dice: 0.930371
Fold 2 - Loss: 0.066702; Accuracy: 0.967530; IoU: 0.874085; Dice: 0.932335
Fold 3 - Loss: 0.054130; Accuracy: 0.971848; IoU: 0.895540; Dice: 0.944579
Fold 4 - Loss: 0.065254; Accuracy: 0.966214; IoU: 0.878956; Dice: 0.935272
Fold 5 - Loss: 0.048540; Accuracy: 0.975437; IoU: 0.907839; Dice: 0.951460
--- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- ---
Aggregated Results - Xception Style K-Fold Evaluation:
Loss: 0.060903
Accuracy: 0.969115
IoU: 0.885423
Dice: 0.938803
Average Number of Epochs: 18.400000
Average Training Time: 17.170000 minutes


![Tensorboard Xception Style Loss Evaluation](./assets/xception-eval/tensorboard-xception-eval.png)
![Tensorboard Xception Style Loss Curve](./assets/xception-eval/tensorboard-loss-curve.png)

## EfficientNetB5

In [14]:
BACKBONE = 'efficientnetb5'
def build_efficientnet_model(hp):
    loss_idx = hp.Choice('loss_idx', values=[0, 1])
    losses = [keras.losses.binary_crossentropy, sm.losses.dice_loss]
    loss = losses[loss_idx]

    optimizer_idx = hp.Choice('optimizer_idx', values=[0, 1])
    optimizers = [keras.optimizers.Adam(), keras.optimizers.SGD()]
    optimizer = optimizers[optimizer_idx]

    model = sm.Unet(BACKBONE, encoder_weights='imagenet', encoder_freeze=True)

    model.compile(
        optimizer=optimizer,
        loss=loss,
        metrics=['accuracy', sm.metrics.IOUScore(), sm.metrics.FScore()]
    )

    return model

In [16]:
from keras.applications.efficientnet_v2 import preprocess_input

def process_dataset(images, labels):
    return preprocess_input(images), labels

for batch_size in [4, 8, 16]:
    project_name = 'efficientnetb5-%d' % batch_size

    training_dataset, validation_dataset = get_data(batch_size)
    training_dataset = training_dataset.map(process_dataset).prefetch(buffer_size=tf.data.AUTOTUNE)
    validation_dataset = validation_dataset.map(process_dataset).prefetch(buffer_size=tf.data.AUTOTUNE)

    tuner = keras_tuner.GridSearch(
        objective='val_loss',
        hypermodel=build_efficientnet_model,
        project_name='models/' + project_name
    )

    tensorboard_cb = keras.callbacks.TensorBoard(log_dir='tensorboard/%s' % (project_name))
    early_stopping_cb = keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True, start_from_epoch=10)
    reduce_lr_cb = keras.callbacks.ReduceLROnPlateau(patience=5, cooldown=5)

    tuner.search(
        training_dataset,
        batch_size=batch_size,
        epochs=50,
        validation_data=validation_dataset,
        callbacks=[tensorboard_cb, early_stopping_cb, reduce_lr_cb]
    )

Found 633 files.
Using 570 files for training.
Using 63 files for validation.
Found 633 files.
Using 570 files for training.
Using 63 files for validation.
Reloading Tuner from ./models/efficientnetb5-4/tuner0.json
Found 633 files.
Using 570 files for training.
Using 63 files for validation.
Found 633 files.
Using 570 files for training.
Using 63 files for validation.
Reloading Tuner from ./models/efficientnetb5-8/tuner0.json
Found 633 files.
Using 570 files for training.
Using 63 files for validation.
Found 633 files.
Using 570 files for training.
Using 63 files for validation.
Reloading Tuner from ./models/efficientnetb5-16/tuner0.json


In [5]:
BACKBONE = 'efficientnetb5'

evaluations = []
histories = []

from keras.applications.efficientnet_v2 import preprocess_input

def process_dataset(images, labels):
    return preprocess_input(images), labels

split, image_names, mask_names = kfold.fold_segmentation_dataset()
for i, (train_idx, test_idx) in enumerate(split):
    batch_size = 4
    project_name = 'efficientnet-evaluation/fold-%d' % (i + 1)
    train_images = image_names[train_idx]
    train_masks = mask_names[train_idx]
    test_images = image_names[test_idx]
    test_masks = mask_names[test_idx]

    print('Splitting for fold %d' % i)
    kfold.copy_segmentation_dataset_to_folds({'images': train_images, 'masks': train_masks}, {'images': test_images, 'masks': test_masks})

    training_dataset, validation_dataset = get_data(batch_size, base_img_dir='segmentation_data/folds/train')
    training_dataset = training_dataset.map(process_dataset).prefetch(buffer_size=tf.data.AUTOTUNE)
    validation_dataset = validation_dataset.map(process_dataset).prefetch(buffer_size=tf.data.AUTOTUNE)

    model = sm.Unet(BACKBONE, encoder_weights='imagenet', encoder_freeze=True)
    model.compile(
        optimizer=keras.optimizers.Adam(),
        loss=sm.losses.dice_loss,
        metrics=['accuracy', sm.metrics.IOUScore(), sm.metrics.FScore()]
    )

    # train the decoder
    model.fit(
        training_dataset,
        validation_data=validation_dataset,
        epochs=3
    )

    # Allow the encoder weights to be updated, at a lower learning rate
    for layer in model.layers:
        layer.trainable = True

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.0001),
        loss=sm.losses.dice_loss,
        metrics=['accuracy', sm.metrics.IOUScore(), sm.metrics.FScore()]
    )

    tensorboard_cb = keras.callbacks.TensorBoard(log_dir='tensorboard/%s' % (project_name))
    early_stopping_cb = keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
    reduce_lr_cb = keras.callbacks.ReduceLROnPlateau(patience=5, cooldown=5)

    history = model.fit(
        training_dataset,
        validation_data=validation_dataset,
        epochs=50,
        callbacks=[tensorboard_cb, early_stopping_cb, reduce_lr_cb]
    )

    test_images = keras.utils.image_dataset_from_directory(
        'segmentation_data/folds/test' + '/images',
        labels=None,
        batch_size=batch_size,
        image_size=(256, 256),
        shuffle=False
    )

    test_masks = keras.utils.image_dataset_from_directory(
        'segmentation_data/folds/test' + '/masks',
        labels=None,
        batch_size=batch_size,
        image_size=(256, 256),
        color_mode='grayscale',
        shuffle=False
    )

    test_dataset = tf.data.Dataset.zip(test_images, test_masks)
    test_dataset = test_dataset.map(augment_validation).prefetch(buffer_size=tf.data.AUTOTUNE)
    test_dataset = test_dataset.map(process_dataset).prefetch(buffer_size=tf.data.AUTOTUNE)

    histories.append(history)
    evaluations.append(model.evaluate(test_dataset))


Splitting for fold 0
Creating fold dirs...
Copying the training data...
Copying the test data...
Found 563 files.
Using 507 files for training.
Using 56 files for validation.
Found 563 files.
Using 507 files for training.
Using 56 files for validation.
115515256/115515256 ━━━━━━━━━━━━━━━━━━━━ 11s 0us/step
Epoch 1/3
127/127 ━━━━━━━━━━━━━━━━━━━━ 251s 2s/step - accuracy: 0.8545 - f1-score: 0.6963 - iou_score: 0.5640 - loss: 0.3037 - val_accuracy: 0.9732 - val_f1-score: 0.9142 - val_iou_score: 0.8421 - val_loss: 0.0858
Epoch 2/3
127/127 ━━━━━━━━━━━━━━━━━━━━ 212s 2s/step - accuracy: 0.9598 - f1-score: 0.9307 - iou_score: 0.8712 - loss: 0.0693 - val_accuracy: 0.9722 - val_f1-score: 0.9398 - val_iou_score: 0.8869 - val_loss: 0.0602
Epoch 3/3
127/127 ━━━━━━━━━━━━━━━━━━━━ 203s 2s/step - accuracy: 0.9628 - f1-score: 0.9419 - iou_score: 0.8908 - loss: 0.0581 - val_accuracy: 0.9564 - val_f1-score: 0.9138 - val_iou_score: 0.8428 - val_loss: 0.0862
Epoch 1/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 278s 2s/ste

In [7]:
np_evals = np.array(evaluations)
evaluation_means = np.mean(np_evals, axis=0)

for i, evaluation in enumerate(np_evals):
    print('Fold %d - Loss: %f; Accuracy: %f; IoU: %f; Dice: %f' % (i + 1, evaluation[0], evaluation[1], evaluation[2], evaluation[3]))

print('--- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- ---')
print('Aggregated Results - EfficientNetB5 K-Fold Evaluation:')
for (mean_score, metric_name) in zip(evaluation_means, ['Loss', 'Accuracy', 'IoU', 'Dice']):
    print('%s: %f' % (metric_name, mean_score))
print('Average Number of Epochs: %f' % 10)
print('Average Training Time: %f minutes' % 30.69)

Fold 1 - Loss: 0.041346; Accuracy: 0.979756; IoU: 0.921621; Dice: 0.958833
Fold 2 - Loss: 0.041139; Accuracy: 0.978774; IoU: 0.921912; Dice: 0.959137
Fold 3 - Loss: 0.042399; Accuracy: 0.977157; IoU: 0.919443; Dice: 0.957630
Fold 4 - Loss: 0.036725; Accuracy: 0.981412; IoU: 0.929642; Dice: 0.963322
Fold 5 - Loss: 0.035588; Accuracy: 0.981755; IoU: 0.931412; Dice: 0.964412
--- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- ---
Aggregated Results - EfficientNetB5 K-Fold Evaluation:
Loss: 0.039440
Accuracy: 0.979771
IoU: 0.924806
Dice: 0.960667
Average Number of Epochs: 18.400000
Average Training Time: 17.170000 minutes


![Tensorboard Xception Style Loss Evaluation](./assets/efficientnet-eval/epoch-loss-time.png)
![Tensorboard Xception Style Loss Curve](./assets/efficientnet-eval/epoch-loss-curve.png)

## Creating The Segmented Classification Dataset

Using the efficientnetb5 model I will create the segmented dataset for classification.

In [3]:
BACKBONE = 'efficientnetb5'
from keras.applications.efficientnet_v2 import preprocess_input
batch_size = 4

def process_dataset(images, labels):
    return preprocess_input(images), labels

training_dataset, validation_dataset = get_data(batch_size, image_size=(512, 512), masks_size=(512, 512), base_img_dir='segmentation_data')
training_dataset = training_dataset.map(process_dataset).prefetch(buffer_size=tf.data.AUTOTUNE)
validation_dataset = validation_dataset.map(process_dataset).prefetch(buffer_size=tf.data.AUTOTUNE)

model = sm.Unet(BACKBONE, encoder_weights='imagenet', encoder_freeze=True)
model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=sm.losses.dice_loss,
    metrics=['accuracy', sm.metrics.IOUScore(), sm.metrics.FScore()]
)

# train the decoder
model.fit(
    training_dataset,
    validation_data=validation_dataset,
    epochs=3
)

# Allow the encoder weights to be updated, at a lower learning rate
for layer in model.layers:
    layer.trainable = True

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0005),
    loss=sm.losses.dice_loss,
    metrics=['accuracy', sm.metrics.IOUScore(), sm.metrics.FScore()]
)

early_stopping_cb = keras.callbacks.EarlyStopping(patience=10)
reduce_lr_cb = keras.callbacks.ReduceLROnPlateau(patience=5, cooldown=5)

history = model.fit(
    training_dataset,
    validation_data=validation_dataset,
    epochs=30,
    callbacks=[reduce_lr_cb, early_stopping_cb]
)

Found 704 files.
Using 634 files for training.


Using 70 files for validation.


2024-08-29 12:56:14.715602: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22456 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:a3:00.0, compute capability: 8.6


Found 704 files.
Using 634 files for training.
Using 70 files for validation.
Epoch 1/3


I0000 00:00:1724936214.680906   63163 service.cc:146] XLA service 0x7f1238025300 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1724936214.680945   63163 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2024-08-29 12:56:56.216256: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-08-29 12:57:00.355306: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8906
I0000 00:00:1724936283.640084   63163 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


159/159 ━━━━━━━━━━━━━━━━━━━━ 218s 712ms/step - accuracy: 0.8057 - f1-score: 0.6550 - iou_score: 0.5094 - loss: 0.3450 - val_accuracy: 0.9232 - val_f1-score: 0.8356 - val_iou_score: 0.7191 - val_loss: 0.1647
Epoch 2/3
159/159 ━━━━━━━━━━━━━━━━━━━━ 36s 225ms/step - accuracy: 0.9417 - f1-score: 0.8728 - iou_score: 0.7794 - loss: 0.1272 - val_accuracy: 0.9624 - val_f1-score: 0.9225 - val_iou_score: 0.8573 - val_loss: 0.0779
Epoch 3/3
159/159 ━━━━━━━━━━━━━━━━━━━━ 37s 227ms/step - accuracy: 0.9469 - f1-score: 0.8895 - iou_score: 0.8053 - loss: 0.1105 - val_accuracy: 0.9790 - val_f1-score: 0.9593 - val_iou_score: 0.9223 - val_loss: 0.0410
Epoch 1/30


E0000 00:00:1724936541.697744   63167 gpu_timer.cc:183] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1724936541.816971   63167 gpu_timer.cc:183] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.


158/159 ━━━━━━━━━━━━━━━━━━━━ 0s 208ms/step - accuracy: 0.9480 - f1-score: 0.8907 - iou_score: 0.8079 - loss: 0.1093

E0000 00:00:1724936644.013411   63167 gpu_timer.cc:183] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1724936644.130250   63167 gpu_timer.cc:183] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.


159/159 ━━━━━━━━━━━━━━━━━━━━ 242s 736ms/step - accuracy: 0.9481 - f1-score: 0.8910 - iou_score: 0.8085 - loss: 0.1090 - val_accuracy: 0.9817 - val_f1-score: 0.9665 - val_iou_score: 0.9353 - val_loss: 0.0337 - learning_rate: 5.0000e-04
Epoch 2/30
159/159 ━━━━━━━━━━━━━━━━━━━━ 37s 227ms/step - accuracy: 0.9597 - f1-score: 0.9163 - iou_score: 0.8481 - loss: 0.0837 - val_accuracy: 0.9829 - val_f1-score: 0.9683 - val_iou_score: 0.9388 - val_loss: 0.0319 - learning_rate: 5.0000e-04
Epoch 3/30
159/159 ━━━━━━━━━━━━━━━━━━━━ 35s 218ms/step - accuracy: 0.9659 - f1-score: 0.9294 - iou_score: 0.8698 - loss: 0.0706 - val_accuracy: 0.9845 - val_f1-score: 0.9711 - val_iou_score: 0.9441 - val_loss: 0.0290 - learning_rate: 5.0000e-04
Epoch 4/30
159/159 ━━━━━━━━━━━━━━━━━━━━ 36s 220ms/step - accuracy: 0.9662 - f1-score: 0.9292 - iou_score: 0.8695 - loss: 0.0708 - val_accuracy: 0.9852 - val_f1-score: 0.9724 - val_iou_score: 0.9466 - val_loss: 0.0277 - learning_rate: 5.0000e-04
Epoch 5/30
159/159 ━━━━━━━━━━━

In [4]:
base_img_dir = './data'
classification_normal = keras.utils.image_dataset_from_directory(
    base_img_dir,
    labels=None,
    batch_size=batch_size,
    image_size=(512, 512),
    shuffle=False
)
classification_tb = keras.utils.image_dataset_from_directory(
    base_img_dir + '/Tuberculosis',
    labels=None,
    batch_size=batch_size,
    image_size=(512, 512),
    shuffle=False
)

dataset = classification_normal.concatenate(classification_tb)
dataset = dataset.map(preprocess_input).prefetch(buffer_size=tf.data.AUTOTUNE)
results = model.predict(dataset)

Found 4200 files.
Found 700 files.
1225/1225 ━━━━━━━━━━━━━━━━━━━━ 46s 30ms/step


In [5]:
DATA_DIR = './segmented_classification'
normal_dir = base_img_dir + '/Normal'
tb_dir = base_img_dir + '/Tuberculosis'
normal_files = [f for f in os.listdir(normal_dir) if os.path.isfile(os.path.join(normal_dir, f))]
tb_files = [f for f in os.listdir(tb_dir) if os.path.isfile(os.path.join(tb_dir, f))]

filenames = normal_files + tb_files

if os.path.exists(os.path.join(DATA_DIR, "out")):
    print("Removing existing output directory...")
    shutil.rmtree(os.path.join(DATA_DIR, "out"))

os.makedirs(os.path.join(DATA_DIR, "out"), exist_ok=True)
out_dir = os.path.join(DATA_DIR, "out")
for pred, fn in zip(results, filenames):
    pred = np.squeeze(pred)
    pred = (pred > 0.5).astype(np.uint8)
    plt.imsave(os.path.join(out_dir, fn), pred, cmap="gray")

Removing existing output directory...


In [6]:
import cv2
import os
import numpy as np

overlayed_out = os.path.join(DATA_DIR, "overlayed_out")

if os.path.exists(overlayed_out):
    print("Removing existing output directory...")
    shutil.rmtree(overlayed_out)

os.makedirs(overlayed_out, exist_ok=True)

def overlay_mask_on_image(image, mask):
    mask = mask > 0.5
    overlay_image = np.zeros_like(image)
    overlay_image[mask] = image[mask]

    return overlay_image

for img_name in filenames:
    base_dir = './data/Normal'
    if 'Tuberculosis' in img_name:
        base_dir = './data/Tuberculosis'
    img_path = os.path.join(base_dir, img_name)
    mask_path = os.path.join(out_dir, img_name)

    original_image = cv2.imread(img_path)
    original_image = cv2.resize(original_image, (512, 512))
    
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    mask = cv2.resize(mask, (512, 512))
    mask = mask / 255.0
    
    overlay_image = overlay_mask_on_image(original_image, mask)

    cv2.imwrite(os.path.join(overlayed_out, img_name), overlay_image)


Removing existing output directory...
